In [1]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
import polars as pl

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    df['county'] = df['county'].fillna('Outside Region')
    
    return df

In [4]:
# network data
df_network = load_network_summary(output_path / 'network/network_results.csv')

In [5]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

In [6]:
#| label: system_summary

# network summary: 'VMT','VHT','total_delay'
vmt = df_network['VMT'].sum()
vht = df_network['VHT'].sum()
delay = df_network['total_delay'].sum()

# Get light rail boardings
df_boardings = pd.read_csv(output_path / 'transit/daily_boardings_by_agency.csv')
# transit boardings
transit_ridership = df_boardings['boardings'].sum()
#Light rail boardings
df_line_boardings = pd.read_csv(output_path / 'transit/transit_line_results.csv')
lr_ridership = df_line_boardings[df_line_boardings['mode']=='r']['boardings'].sum()

# mode share
df_trip = pd.read_csv(output_path / 'agg/dash/mode_share_county.csv')
df = df_trip['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"]['trexpfac'].sum()/df

# emission
df_emissions = pd.read_csv(output_path / 'emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df['total_daily_tons'].sum()

run_summary2 = pd.DataFrame({
    'VMT': [vmt],
    'VHT': [vht],
    'Delay': [delay],
    'Transit Boardings': [transit_ridership],
    'Light Rail Boardings': [lr_ridership],
    '% Transit': [transit_share],
    'CO2e': [CO2e]
})

run_summary2.style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit']).\
            hide(axis="index")

VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
"99,172,787","3,127,777","331,045","1,589,030","503,023",4.8%,"6,524"


In [7]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

In [8]:
#| label: total_vmt_vht_delay_by_county

df_vmt = df_network.reset_index().groupby('county',observed=True)['VMT'].sum()
df_vht = df_network.reset_index().groupby('county',observed=True)['VHT'].sum()
df_delay = df_network.reset_index().groupby('county',observed=True)['total_delay'].sum()

df = pd.concat([df_vmt,df_vht,df_delay], axis=1)
df = df[df.index!='Outside Region']
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)

df.loc['Total',] = df.sum()
total_vmt = df.loc['Total','VMT']

df.style.format('{:,.0f}')

,VMT,VHT,Total Delay Hours
county,,,
King,"50,690,443","1,640,079","210,837"
Kitsap,"5,454,221","161,790","4,831"
Pierce,"22,348,574","697,619","56,994"
Snohomish,"20,250,448","619,284","58,164"
Total,"98,743,686","3,118,773","330,826"


In [9]:
#| label: transit
transit = pd.read_csv(output_path / 'transit/daily_boardings_by_agency.csv')

df = transit[['agency_name','boardings']].set_index('agency_name').copy()
df.loc['Total',:] = df['boardings'].sum()
df.rename(columns={'boardings': 'Daily Boardings'}, inplace=True)


df.style.format('{:,.0f}')


,Daily Boardings
agency_name,
King County Metro,"802,085"
Sound Transit,"540,922"
Community Transit,"95,682"
Pierce Transit,"71,893"
Kitsap Transit,"38,644"
Washington Ferries,"21,517"
Everett Transit,"18,288"
Total,"1,589,030"


In [10]:
#| label: mode_share
mode_share = pd.read_csv(output_path / 'agg/dash/mode_share_county.csv')

df = mode_share.groupby('mode')['trexpfac'].sum().reset_index().set_index('mode')
df['Mode Share'] = (df['trexpfac']/ (mode_share['trexpfac'].sum()))
df.loc['Total', :] = df.sum(axis=0)
df['Mode Share'] = (df['Mode Share'] * 100).round(1)
df['Mode Share'] = df['Mode Share'].astype(str) + '%'
df['trexpfac'] = df['trexpfac'].apply(lambda x: f"{int(round(x)):,}")
df.rename(columns={'trexpfac': 'Total Person Trips'}, inplace=True)
df

,Total Person Trips,Mode Share
mode,,
Bike,"388,281",1.8%
HOV2,"4,362,194",19.8%
HOV3+,"2,744,504",12.4%
SOV,"8,663,747",39.3%
School Bus,"303,236",1.4%
TNC,"126,306",0.6%
Transit,"1,056,537",4.8%
Walk,"4,421,611",20.0%
Total,"22,066,416",100.0%
